# Optimasi Vehicle Routing Problem <br> Alur Pembuangan Sampah Rayon Surabaya Pusat
<b> Kelompok 08 </b> <br>
Nama Anggota :
1. Nida Aulia Amartika			(5026221095)
2. Isaura Qinthara Heriswan		(5026221146)
3. Devy Relliani Saffiyah		(5026221189)


# Exploratory Data Analysis

In [8]:
%pip install scikit-learn
%pip install folium

Note: you may need to restart the kernel to use updated packages.

   ------------- -------------------------- 1/3 [branca]
   -------------------------- ------------- 2/3 [folium]
   -------------------------- ------------- 2/3 [folium]
   -------------------------- ------------- 2/3 [folium]
   -------------------------- ------------- 2/3 [folium]
   -------------------------- ------------- 2/3 [folium]
   -------------------------- ------------- 2/3 [folium]
   -------------------------- ------------- 2/3 [folium]
   ---------------------------------------- 3/3 [folium]

Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
import numpy as np
import random
from sklearn.cluster import KMeans
import folium
import math

In [17]:
df = pd.read_csv('new_data.csv')
df

,No,LPS/Depo,Alamat,Latitude,Longitude
0,1,Demak (Kali Butuh),Jalan Demak,-7.253606,112.720422
1,2,Pringadi,Jalan Pringadi,-7.252544,112.733072
2,3,Penghela,Jalan Penghela,-7.248233,112.733186
3,4,Sulung Kali,Jalan Sulung Kali,-7.244014,112.742173
4,5,Dupak,Jalan Babatan Dupak,-7.244836,112.727439
5,6,Simolawang,Jalan Simolawang,-7.237522,112.753542
6,7,Pasar Kapasan,Jalan Simolawang Baru I,-7.239633,112.750247
7,8,Tambak Rejo,Jalan Kenjeran (depan Makam Rangkah),-7.243069,112.760314
8,9,Simpang Dukuh,Jalan Simpang Dukuh,-7.260503,112.742111
9,10,Pasar Genteng,Jalan Genteng Besar,-7.258242,112.740377


In [18]:
pool = (-7.26053139, 112.692194)   # Pool Tanjungsari
tpa  = (-7.23739100, 112.60943600) # TPA Benowo

In [ ]:
m1 = folium.Map(location=pool, zoom_start=12)
folium.Marker(location=pool, popup='Pool Tanjungsari', icon=folium.Icon(color='red')).add_to(m1)
folium.Marker(location=tpa, popup='TPA Benowo', icon=folium.Icon(color='green')).add_to(m1)
for _, row in df.iterrows():
    folium.Marker(location=(row['Latitude'], row['Longitude']), popup=row['LPS/Depo']).add_to(m1)
m1  # Preview map before VRP

In [20]:
n_vehicles = 11
coords = df[['Latitude','Longitude']].values
kmeans = KMeans(n_clusters=n_vehicles, random_state=42).fit(coords)
df['cluster'] = kmeans.labels_

In [21]:
colors = ['blue', 'purple', 'orange', 'yellow', 'lightred', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'pink', 'lightblue']

# Create a new map centered at the pool
m_cluster = folium.Map(location=pool, zoom_start=12)

# Add depot and TPA markers
folium.Marker(location=pool, popup='Pool Tanjungsari', icon=folium.Icon(color='red')).add_to(m_cluster)
folium.Marker(location=tpa, popup='TPA Benowo', icon=folium.Icon(color='green')).add_to(m_cluster)

# Add markers for each point, colored by cluster
for _, row in df.iterrows():
    folium.Marker(location=(row['Latitude'], row['Longitude']),
                  popup=f"{row['LPS/Depo']} (Cluster {row['cluster']})",
                  icon=folium.Icon(color=colors[row['cluster'] % len(colors)])).add_to(m_cluster)

# Display the map
m_cluster

C:\Users\Devy Relliani\AppData\Local\Temp\ipykernel_10488\1510036970.py:14: UserWarning: color argument of Icon should be one of: {'lightblue', 'lightred', 'lightgreen', 'black', 'lightgray', 'darkred', 'white', 'darkblue', 'cadetblue', 'darkgreen', 'green', 'red', 'darkpurple', 'purple', 'orange', 'blue', 'gray', 'beige', 'pink'}.
  icon=folium.Icon(color=colors[row['cluster'] % len(colors)])).add_to(m_cluster)


In [25]:
def haversine(a, b):
    R = 6371e3
    φ1, φ2 = math.radians(a[0]), math.radians(b[0])
    Δφ = math.radians(b[0] - a[0])
    Δλ = math.radians(b[1] - a[1])
    x = math.sin(Δφ/2)**2 + math.cos(φ1)*math.cos(φ2)*math.sin(Δλ/2)**2
    return R * 2 * math.asin(math.sqrt(x))

In [26]:
def route_distance(route):
    dist = 0
    prev = pool
    for pt in route:
        dist += haversine(prev, pt)
        prev = pt
    dist += haversine(prev, tpa) + haversine(tpa, pool)
    return dist

In [27]:
def order_crossover(p1, p2):
    size = len(p1)
    a, b = sorted(random.sample(range(size), 2))
    c1 = [None]*size
    c2 = [None]*size
    c1[a:b], c2[a:b] = p1[a:b], p2[a:b]
    def fill(child, parent):
        pos = b
        for gene in parent[b:] + parent[:b]:
            if gene not in child:
                if pos >= size: pos = 0
                child[pos] = gene
                pos += 1
    fill(c1, p2)
    fill(c2, p1)
    return c1, c2

In [29]:
def mutate(ind):
    if len(ind) > 1:
        i, j = random.sample(range(len(ind)), 2)
        ind[i], ind[j] = ind[j], ind[i]

In [30]:
def ga_cluster(points, pop_size=150, gens=800, elite_size=1):
    n = len(points)
    if n < 2:
        return points

    # initialize population
    population = [random.sample(range(n), n) for _ in range(pop_size)]

    for _ in range(gens):
        # compute fitness for roulette selection
        fitness = [1/(route_distance([points[i] for i in ind]) + 1) for ind in population]
        total = sum(fitness)
        probs = [f/total for f in fitness]

        # select elites (best individuals) to carry forward
        sorted_pop = sorted(
            population,
            key=lambda ind: route_distance([points[i] for i in ind])
        )
        new_pop = [sorted_pop[i].copy() for i in range(elite_size)]

        # fill the rest of new_pop by crossover + mutation
        while len(new_pop) < pop_size:
            # select two parents
            p1, p2 = random.choices(population, probs, k=2)
            # crossover
            if random.random() < 0.8:
                c1, c2 = order_crossover(p1, p2)
            else:
                c1, c2 = p1[:], p2[:]
            # mutate
            if random.random() < 0.2:
                mutate(c1)
            if random.random() < 0.2:
                mutate(c2)

            new_pop.append(c1)
            if len(new_pop) < pop_size:
                new_pop.append(c2)

        population = new_pop

    # return the best route as actual coordinates
    best = min(
        population,
        key=lambda ind: route_distance([points[i] for i in ind])
    )
    return [points[i] for i in best]

In [ ]:
m2 = folium.Map(location=pool, zoom_start=12)
folium.Marker(location=pool, popup='Pool Tanjungsari', icon=folium.Icon(color='red')).add_to(m2)
folium.Marker(location=tpa, popup='TPA Benowo', icon=folium.Icon(color='green')).add_to(m2)

# Colors already defined above - no need to redefine

coord_to_name = {
    (row['Latitude'], row['Longitude']): row['LPS/Depo']
    for _, row in df.iterrows()
}
coord_to_name[pool] = "Pool Tanjungsari"
coord_to_name[tpa]  = "TPA Benowo"

for vid in range(n_vehicles):
    # get the raw coordinates in this cluster
    cluster_pts = coords[kmeans.labels_ == vid]
    if len(cluster_pts) == 0:
        continue

    # GA wants a Python list of tuple‐points
    pts_list = [tuple(pt) for pt in cluster_pts]

    # find best ordering of stops
    best_route = ga_cluster(pts_list)

    # build the full trip for plotting
    full_route = [pool] + best_route + [tpa, pool]

    # 1) draw it on the map
    folium.PolyLine(
        locations=full_route,
        color=colors[vid % len(colors)],
        weight=2.5, opacity=0.8,
        popup=f'Vehicle {vid+1}'
    ).add_to(m2)

    # 2) compute the true total distance via route_distance(best_route)
    #    (it already adds pool→stops→TPA→pool)
    total_dist_km = route_distance(best_route) / 1000

    # 3) map coords back to names
    path_names = [coord_to_name[c] for c in full_route]
    print(f"Vehicle {vid+1}: " + " → ".join(path_names))
    print(f"  Total distance: {total_dist_km:.2f} km\n")

# finally show the map
m2

In [ ]:
colors = ['blue', 'purple', 'orange', 'yellow', 'lightred', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'pink', 'lightblue']

coord_to_name = {
    (row['Latitude'], row['Longitude']): row['LPS/Depo']
    for _, row in df.iterrows()
}
coord_to_name[pool] = "Pool Tanjungsari"
coord_to_name[tpa]  = "TPA Benowo"

for vid in range(n_vehicles):
    # get the raw coordinates in this cluster
    cluster_pts = coords[kmeans.labels_ == vid]
    if len(cluster_pts) == 0:
        continue

    # GA wants a Python list of tuple‐points
    pts_list = [tuple(pt) for pt in cluster_pts]

    # find best ordering of stops
    best_route = ga_cluster(pts_list)

    # build the full trip for plotting
    full_route = [pool] + best_route + [tpa, pool]

    # 1) draw it on the map
    folium.PolyLine(
        locations=full_route,
        color=colors[vid % len(colors)],
        weight=2.5, opacity=0.8,
        popup=f'Vehicle {vid+1}'
    ).add_to(m2)

    # 2) compute the true total distance via route_distance(best_route)
    #    (it already adds pool→stops→TPA→pool)
    total_dist_km = route_distance(best_route) / 1000

    # 3) map coords back to names
    path_names = [coord_to_name[c] for c in full_route]
    print(f"Vehicle {vid+1}: " + " → ".join(path_names))
    print(f"  Total distance: {total_dist_km:.2f} km\n")

# finally show the map
m2

Vehicle 1: Pool Tanjungsari → Sawahan → Pasar Kembang → Kedondong → Pandegiling → Rumah Sakit Darmo / Ketampon → Kaiser → TPA Benowo → Pool Tanjungsari
  Total distance: 32.22 km

Vehicle 2: Pool Tanjungsari → Alas Malang → Kendung → Babat Jerawat → TPA Benowo → Pool Tanjungsari
  Total distance: 19.83 km

Vehicle 3: Pool Tanjungsari → Gebang Putih → Keputih → Semolowaru Bahari → TPA Benowo → Pool Tanjungsari
  Total distance: 46.01 km

Vehicle 4: Pool Tanjungsari → Demak (Kali Butuh) → Kedung Anyar → Pringadi → Pasar Genteng → Makam Peneleh → Sulung Kali → Panghela → Penghela → Dupak → TPA Benowo → Pool Tanjungsari
  Total distance: 31.67 km

Vehicle 5: Pool Tanjungsari → Jagir → Gayungsari → 3R Jambangan → TPA Benowo → Pool Tanjungsari
  Total distance: 38.69 km

Vehicle 6: Pool Tanjungsari → Dinoyo → Keputran Selatan → Kayun → Kayon → Srikana → Gubeng → Pacar Keling → Legundi Anggrek → Simpang Dukuh → TPA Benowo → Pool Tanjungsari
  Total distance: 37.16 km

Vehicle 7: Pool Tanjungs